In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Cargar los datos desde los archivos
df_estudiantes = pd.read_csv("./archivos-bd/BaseAprenderEstudiantes6Primaria2022-MuestralLyM.csv", sep=';')
df_referencias = pd.read_excel("./archivos-bd/EstudiantesAPRENDER2022PrimariaMuestral-DiccionarioDeVariables.xlsx", sheet_name="etiquetas de variables", header=1)
df_valores_variables = pd.read_excel("./archivos-bd/EstudiantesAPRENDER2022PrimariaMuestral-DiccionarioDeVariables.xlsx", sheet_name="valores de variables", header=1)

### Análisis Exploratorio de Datos

In [ ]:
print(df_estudiantes.shape)
print(df_referencias.shape)
print(df_valores_variables.shape)

In [ ]:
df_estudiantes.head()

In [ ]:
df_referencias.head()

In [ ]:
df_valores_variables.head(28)

In [ ]:
# Transformar el dataframe de referencias a una Serie para poder buscar por el nombre de la variable más facil
df_referencias = df_referencias.set_index(df_referencias.columns[0]).squeeze()

In [ ]:
# Rellenar las columnas que tienen valores NaN por la variable correspondiente
for index, _ in enumerate(df_valores_variables["Variable"]):
    if index != 0:
        # Si estoy en un valor NaN y el anterior tiene un valor significa que debe tener el mismo valor que el anterior
        if pd.isnull(df_valores_variables["Variable"][index]) and not pd.isnull(df_valores_variables["Variable"][index-1]):
            df_valores_variables["Variable"][index] = df_valores_variables["Variable"][index-1]

df_valores_variables.head(30)

In [ ]:
# Recorremos las columnas para saber si hay elementos nulos en las columnas del dataframe
print(df_estudiantes.isnull().sum()) # Trunca el resultado

# Según este análisis ninguna columna tienen valores nulos pero analizando un poco mas a fondo observamos
# que hay columnas que tienen como valor un string vacio o un espacio " " por lo que se debe considerar
# como se tratarán estos caso
print(df_estudiantes["ap04"].unique())

#! Se debe tener especial cuidado porque en la misma columna hay variables del tipo int y string
#! que representan lo mismo

In [ ]:
# Definir una funcion para saber que indices nos sirven y que valor de refencia tienen
def buscar_referencias(nombre_referencia):
    return df_referencias[df_referencias.str.contains(nombre_referencia)]


## Practica para todas las muestras

### 1 - Graficar, en un mismo plano, la proporción de muestras con y sin acceso a internet.

In [ ]:
# Para saber que variables hacen refencia a "Internet" utilizamos la función buscar_referencias
referencias_a_internet = buscar_referencias("Internet")

print(referencias_a_internet)

print(referencias_a_internet.values[0])
print(referencias_a_internet.index[0])

# Por lo que determinamos que la variable que analizaremos será "ap12d": "¿Disponés de los siguientes objetos o servicios en el lugar donde vivís? [Conexión a Internet]""
print(df_estudiantes["ap12d"].unique())

#### Conversion y limpieza de datos

Mediante la observación manual de los datos, nos dimos cuenta de que el dataset no tiene de por sí variables categóricas porque todas estan mapeadas a valores enteros, sin embargo, como se verá más adelante, hay columnas del dataframe que contiene espacios y que tienen numeros y strings que representan lo mismo

In [ ]:
def normalizar_categoricas(param):
    if isinstance(param, str):
        if param.isspace() or param == "":
            return int(param.replace(" ", "-9"))
        else:
            return int(param)
    else:
        return param

df_estudiantes["ap12d"] = df_estudiantes["ap12d"].map(normalizar_categoricas)

df_estudiantes["ap12d"].unique()

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

etiquetas = {
    1: 'Sí',
    2: 'No'
}

conteo_valores = df_estudiantes['ap12d'].value_counts()
conteo_valores = conteo_valores.drop(conteo_valores.index[[1, 3, 4]])

print(conteo_valores)

# Obtener los valores y etiquetas
valores = conteo_valores.values
etiquetas_pie = [etiquetas[valor] for valor in conteo_valores.index]

# Crear el gráfico de pie
plt.title(
    'Proporción de estudiantes con acceso a internet',
    fontdict={"fontsize": 16})
handles = plt.pie(
    valores, 
    labels=etiquetas_pie, 
    autopct='%1.1f%%', 
    startangle=90,
    textprops={"fontsize": 13})
fig.set_facecolor('white')

plt.show()


### 2 - Informar el porcentaje de estudiantes mayores de 13 años.

In [ ]:
referencias_a_años = buscar_referencias("años")

print(referencias_a_años)

# Por lo que determinamos que la variable que analizaremos será "ap01": "¿Cuántos años tenés?"
print(df_referencias["ap01"])

print(df_estudiantes["ap01"].unique())

In [ ]:
df_estudiantes["ap01"] = df_estudiantes["ap01"].map(normalizar_categoricas)

df_estudiantes["ap01"].unique()

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

etiquetas = {
    'menor13': 'Alumnos menores de 13 años',
    'mayor13': 'Alumnos de 13 o más años'
}

conteo_valores = df_estudiantes["ap01"].value_counts().sort_index()

# Obtener los valores y etiquetas
valores = pd.Series([conteo_valores.iloc[0:6].sum(), conteo_valores.iloc[6:].sum()], index=["menor13", "mayor13"])
print(valores)
etiquetas_pie = [etiquetas[valor] for valor in valores.index]

plt.title(
    'Porcentaje de estudiantes mayores de 13 años',
    fontdict={"fontsize": 16})

# Crear el gráfico de pie
handles = plt.pie(
    valores, 
    labels=etiquetas_pie, 
    autopct='%1.1f%%',
    startangle=90,
    textprops={"fontsize": 13})
fig.set_facecolor('white')

plt.show()

### 3 -  Graficar, en un mismo plano, las proporciónes de muestras de las 10 jurisdicciones con mayor cantidad de estudiantes.

In [ ]:
print(df_referencias["jurisdiccion"])

print(df_estudiantes["jurisdiccion"].unique())

# Agrupar con jurisdiccion y sumar para ver si se consideran todos los valores, la suma debe dar igual a 128482
df_estudiantes["jurisdiccion"].value_counts().values.sum()

In [ ]:
# Armar una Serie que contenga el valor de la provincia como key y el nombre como value
df_jurisdiccion = df_valores_variables[df_valores_variables["Variable"] == "jurisdiccion"]
df_jurisdiccion_sin_columna_variable = df_jurisdiccion.drop("Variable", axis=1)
valores_etiqueta = df_jurisdiccion_sin_columna_variable.set_index(df_jurisdiccion_sin_columna_variable.columns[0]).squeeze()

# Acortar nombre CABA
valores_etiqueta[2] = "CABA"

valores_etiqueta.values

In [ ]:
# Reemplazar los valores por su correspondiente provincia
cantidad_estudiantes_por_jurisdiccion = df_estudiantes["jurisdiccion"].value_counts()

cantidad_estudiantes_por_jurisdiccion.index = cantidad_estudiantes_por_jurisdiccion.index.map(valores_etiqueta)

# Tomar las 10 jurisdicciones con mayor poblacion
cantidad_estudiantes_por_jurisdiccion = cantidad_estudiantes_por_jurisdiccion.iloc[:10]

cantidad_estudiantes_por_jurisdiccion


In [ ]:
# Crear un gráfico de barras horizontales
plt.barh(cantidad_estudiantes_por_jurisdiccion.index, cantidad_estudiantes_por_jurisdiccion.values)

# Etiquetas y título
plt.xlabel('Cantidad de Estudiantes')
plt.ylabel('Jurisdicción')
plt.title('Cantidad de Estudiantes por Jurisdicción')

# Mostrar el gráfico
plt.show()

### 4 - Graficar, en un mismo plano, la proporción de muestras con residencia rural y urbana.

In [ ]:
# Encontrar que variable hacia referencia a si el ámbito era rural o urbano fue más dificil porque no había una relación directa
# en el dataframe de referencia por lo que su obtención se hizo mediante la observación manual

print(df_valores_variables[df_valores_variables["Variable"] == "ambito"])
print(df_estudiantes["ambito"].unique())

In [ ]:
fig, ax = plt.subplots()

etiquetas = {
    1: "Rural",
    2: "Urbano"
}

plt.title('Proporción de estudiantes con residencia rural y urbana')

conteo_valores = df_estudiantes['ambito'].value_counts()

# Obtener los valores y etiquetas
valores = conteo_valores.values
etiquetas_pie = [etiquetas[valor] for valor in conteo_valores.index]

# Crear el gráfico de pie
handles = plt.pie(valores, labels=etiquetas_pie, autopct='%1.1f%%', startangle=90)
fig.set_facecolor('white')

# plt.legend(zip(etiquetas.values(), valores), title="Etiquetas y valores", loc="upper right")

plt.show()

## Para las muestras correspondientes a la provincia de Santa Fe:

### 6 - Informar el promedio de los puntajes en matemática y lengua.

In [65]:
# Se define el dataframe para solamente quedarnos con los estudiantes que sean de la provincia de santa fe
df_estudiantes_santafesinos = df_estudiantes[df_estudiantes["jurisdiccion"] == 82]
df_estudiantes_santafesinos

,jurisdiccion,ID1,seccion,idalumno,sector,ambito,ap01,ap02,ap03,ap04,...,lpuntaje,mpuntaje,NSE_puntaje,NSE_nivel,edadA_junio2022,migracion,sobreedad,Nivel_Ed_Madre,Nivel_Ed_Padre,Nivel_Ed_Persona_Resp
3520,82,358050045108803,2,9,1,2,-9,,,,...,"426,00357","481,41885",,,,,,,,
3521,82,358050045108803,2,10,1,2,-9,,,,...,"499,49847","554,62329",,,,,,,,
3522,82,358050045108803,1,3,1,2,-9,,,,...,,"452,96954",,,,,,,,
3523,82,302100467571030,2,32,2,2,-9,,,,...,"442,3382","467,93829",,,,,,,,
3524,82,856320078746888,3,2,1,2,-9,,,,...,"591,84363","556,33777",,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128415,82,320065608032006,3,1,1,2,5,1,-9,1,...,"380,58011","412,80228","-,82300133",2,5,2,4,3,3,
128416,82,667700902286667,5,9,1,2,5,11,2,1,...,,,,,5,2,4,3,,
128417,82,320065608032006,1,11,1,2,5,3,1,11,...,"352,95807",,"-,73130059",2,5,1,4,5,,
128418,82,460600077093164,3,6,1,2,5,6,2,1,...,"413,4458","399,13705","-1,523313",1,5,2,4,4,4,
